In [1]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#Extracting feautures from soundfile
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        audio_data = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(audio_data))
        result = np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(audio_data, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#emotions in RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#emotions that we are gonna observe
observed_emotions = ['calm','happy','sad','angry']

In [4]:
#loading the data and extracting features/emotion labels from files
def load_data(test_size = 0.2):
    x,y = [],[]
    for file in glob.glob("C:\\Users\\TESTER\\Desktop\\python_proj\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file,mfcc=True,chroma=True,mel=True)
        x.append(feature)
        y.append(emotion)
    
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
#spliting dataset in trainnig and validation set
x_train, x_test, y_train, y_test = load_data(test_size = 0.25)

print((x_train.shape[0],x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')

(576, 192)
Features extracted: 180


In [12]:
#using a multi layer perceptron 

model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (300,), verbose = True, learning_rate = 'adaptive', max_iter = 10000, solver = 'lbfgs')

model.fit(x_train,y_train)


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=10000, solver='lbfgs',
              verbose=True)

In [15]:
y_pred=model.predict(x_test)
print(x_test)

[[-6.01416138e+02  6.33363152e+01 -1.16155615e+01 ...  7.30560214e-06
   4.64440200e-06  1.78248013e-06]
 [-7.87200317e+02  5.97919350e+01  2.11022377e+01 ...  4.30602057e-07
   4.01216226e-07  4.06060309e-07]
 [-5.60616272e+02  5.78973389e+01 -8.86894703e+00 ...  6.46989676e-04
   3.31015413e-04  1.85833647e-04]
 ...
 [-4.44762115e+02  2.36135807e+01 -1.48351059e+01 ...  8.65290698e-04
   5.16015454e-04  3.73600313e-04]
 [-5.39169983e+02  4.86916466e+01 -2.04568434e+00 ...  2.49171353e-05
   1.78593928e-05  7.42584143e-06]
 [-6.95250427e+02  4.46738052e+01 -7.46818161e+00 ...  2.15334803e-05
   1.84785295e-05  1.01300748e-05]]


In [14]:

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.62%


In [18]:

file = "C:\\Users\\TESTER\\Desktop\\python_proj\\speech-emotion-recognition-ravdess-data\\Actor_12\\03-01-02-01-02-01-12.wav"

emotion = model.predict(extract_feature(file,mfcc=True,chroma=True,mel=True).reshape(1,-1))
print(emotion)

['calm']
